# Karatsuba
## Algorithme naïf

In [3]:
let produit_naif p1 p2 =
  let n1 = Array.length p1 in
  let n2 = Array.length p2 in
  let p3 = Array.make (n1+n2-1) 0 in
  for i = 0 to n1-1 do
    for j = 0 to n2 -1 do
      p3.(i+j) <- p3.(i+j) + p1.(i)*p2.(j)
    done
  done;
  p3
;;

val produit_naif : int array -> int array -> int array = <fun>


Complexité en $O(n_1 n_2)$.

## Algorithme de Karatsuba

In [11]:
let somme p1 p2 =
  let n1 = Array.length p1 in
  let n2 = Array.length p2 in
  let p3 = Array.make (max n1 n2) 0 in
  for i = 0 to n1-1 do
    p3.(i) <- p1.(i)
  done;
  for i = 0 to n2-1 do
    p3.(i) <- p3.(i) + p2.(i)
  done;
  p3
;;

val somme : int array -> int array -> int array = <fun>


Complexité de `somme p1 p2` en $O(n_1+n_2)$ où $n_i$ longueur de $p_i$.

In [5]:
let difference p1 p2 =
  let n1 = Array.length p1 in
  let n2 = Array.length p2 in
  let p3 = Array.make (max n1 n2) 0 in
  for i = 0 to n1-1 do
    p3.(i) <- p1.(i)
  done;
  for i = 0 to n2-1 do
    p3.(i) <- p3.(i) - p2.(i)
  done;
  p3
;;

val difference : int array -> int array -> int array = <fun>


Idem : complexité linéaire en la taille des tableaux.

In [12]:
let decoupe p =
  (* Renvoie p0, p1 tq p = p0 + X^m p1 *)
  let n = Array.length p in
  let m = (n+1) / 2 in
  let p0 = Array.make m 0 in
  let p1 = Array.make (n-m) 0 in
  for i = 0 to m-1 do
    p0.(i) <- p.(i)
  done;
  for i = m to n-1 do
    p1.(i-m) <- p.(i)
  done;
  p0, p1
;;

val decoupe : int array -> int array * int array = <fun>


Complexité linéaire en la taille de p

In [7]:
let rec karatsuba p q =
  (* On suppose p et q de même longueur *)
  match Array.length p with
  | 0 -> [||]
  | 1 -> [|p.(0)*q.(0)|]
  | n -> 
    let p0, p1 = decoupe p in
    let q0, q1 = decoupe q in
    let pq = Array.make (2*n -1) 0 in

    let p0q0 = karatsuba p0 q0 in
    let p1q1 = karatsuba p1 q1 in
    let produit_croise =
      karatsuba (somme p0 p1) (somme q0 q1)
    in
    let milieu =
      difference produit_croise (somme p0q0 p1q1)
    in
    let m = (n+1)/2 in
    for i = 0 to m-1 do
      pq.(i) <- p0q0.(i)
    done;
    for i = 0 to m-1 do
      pq.(i+m) <- milieu.(i)
    done;
    for i = 0 to n-m-1 do
      pq.(2*m+i) <- p1q1.(i)
    done;
    pq
;;

val karatsuba : int array -> int array -> int array = <fun>


## Étude de la complexité

Soit $n$ la longueur des tableaux ; si $n \geq 2$ alors :
- on effectue deux découpes en temps $O(n)$
- on crée le tableau `pq` en temps $O(n)$
- on effectue deux **appels récursifs** sur des tableaux de taille $\lceil n/2 \rceil$ et $\lfloor n/2 \rfloor$
- on effectue deux sommes sur des tableaux de taille $\lceil n/2 \rceil$, en temps $O(n)$
- on effectue un **appel récursif** sur des tableaux de taille $\lceil n/2 \rceil$
- on effectue une différence avec des tableaux de taille au plus $2\lceil n/2 \rceil$, en temps $O(n)$
- on alimente le tableau `pq` en temps $O(n)$

Finalement, en notant $C(n)$ le temps de calcul dans le pire des cas pour un tableau de taille $n \geq 2$, alors
$$C(n) \leq 2 C \left( \lceil n/2 \rceil \right) + C \left( \lfloor n/2 \rfloor \right) + O(n)$$

Quitte à remplacer $C(n)$ par $\max \{ C(k) \mid k \leq n \}$, on peut supposer $C$ croissante.

Soit $r \in \mathbb{N}$ ; si $r \geq 1$,

$C(2^r) \leq 3 C(2^{r-1}) + \alpha 2^r$, où $\alpha$ est une constante.

En divisant par $3^r$,
$\frac{C(2^r)}{3^r} \leq \frac{C(2^{r-1})}{3^{r-1}} + \alpha \left(\frac{2}{3}\right)^r$

$$\sum_{r=1}^p \frac{C(2^r)}{3^r} - \frac{C(2^{r-1})}{3^{r-1}} \leq \sum_{r=1}^p \alpha \left(\frac{2}{3}\right)^r$$

$$\frac{C(2^p)}{3^p} - C(1)\leq \alpha \frac{2}{3}\frac{1-(2/3)^p}{1-2/3}$$

$$\frac{C(2^p)}{3^p} - C(1)\leq 2\alpha (1-(2/3)^p)\leq 2 \alpha = cte$$ 

$$C(2^p) \leq 3^p \beta \text{ où } \beta \text{ est une constante}$$

Soit $n \geq 2$, posons $p = \lceil \log_2 n \rceil$, de sorte que $p-1 < \log_2 n \leq p$.

Alors $n \leq 2^p$, donc $C(n) \leq \beta 3^p = \beta 3^{\lceil \log_2 n \rceil}$.

**Conclusion :** $C(n) = O(2^{\log_2(3)\log_2 n})$ ie  $C(n) = O(n^{\log_2(3)})$

Algorithme naïf : complexité en $O(n^2)$, a-t-on fait mieux ?

Comme 3 < 4, $\log_2 3 < \log_2 4$ i.e $\log_2 3 < 2$.